In [2]:
import torch
import torch.nn as nn

In [5]:
class attentionSH(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,kqv_bias=False):
        super().__init__()
        self.W_Q = torch.nn.Linear(d_in,d_out,bias=kqv_bias)
        self.W_K = torch.nn.Linear(d_in,d_out,bias=kqv_bias)
        self.W_V = torch.nn.Linear(d_in,d_out,bias=kqv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        #self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))
        self.mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)


    def forward(self,x):
        b,context_length,dimention = x.shape
        queries = self.W_Q(x) # (b , c , d_out)
        keys = self.W_K(x)    # (b , c , d_out) 
        values = self.W_V(x)  # (b , c , d_out)
        d_k = queries.shape[-1]
        attention_score = queries @ keys.transpose(1,2) # (b , c , c)
        attention_score = attention_score.masked_fill(self.mask.bool(),-torch.inf) # (b , c , c)
        attention_weight = torch.softmax(attention_score/d_k**0.5,dim=-1) # (b , c , c)
        attention_weight = self.dropout(attention_weight) # (b , c , c)
        context_vector = attention_weight @ values # (b , c , d_out)
        return (context_vector , attention_weight , attention_score , queries , keys , values) 
        #return attention_weight



In [6]:
torch.manual_seed(123)
din = 3
dout = 4
context_length = 5
dropout = 0
x = torch.randn(context_length,din)
x_batch = torch.stack((x,x,x))
sa = attentionSH(din,dout,context_length,dropout)
context_vector , attention_weight , attention_score , queries , keys , values  = sa(x_batch)
context_vector
#attention_weight.shape
#attention_score.shape
#queries


tensor([[[ 0.0864, -0.1132,  0.0990,  0.0047],
         [-0.1818, -0.0137,  0.3056,  0.0680],
         [-0.1301,  0.1551,  0.4405,  0.0650],
         [-0.0764,  0.1086,  0.3719,  0.0507],
         [-0.0239,  0.1099,  0.2289,  0.0246]],

        [[ 0.0864, -0.1132,  0.0990,  0.0047],
         [-0.1818, -0.0137,  0.3056,  0.0680],
         [-0.1301,  0.1551,  0.4405,  0.0650],
         [-0.0764,  0.1086,  0.3719,  0.0507],
         [-0.0239,  0.1099,  0.2289,  0.0246]],

        [[ 0.0864, -0.1132,  0.0990,  0.0047],
         [-0.1818, -0.0137,  0.3056,  0.0680],
         [-0.1301,  0.1551,  0.4405,  0.0650],
         [-0.0764,  0.1086,  0.3719,  0.0507],
         [-0.0239,  0.1099,  0.2289,  0.0246]]], grad_fn=<UnsafeViewBackward0>)